In [1]:
import os
import inspect
import enum

In [2]:
def load_api():
    """ Load the api and return a module object """

    import sys
    import importlib
    from pathlib import Path

    #####################################################################################################################
    if os.name == 'nt':
        ig_dir = Path("D:\\Development\\Ignis\\build\\Release\\api\\ignis")
    else:
        ig_dir = Path("/home/yazici/dev/Ignis/build/Release/api/ignis") # <----------- Adapt this to your need.
    #####################################################################################################################
    # The directory has to contain a shared library 'pyignis*'.
    # The actual name of the shared library depends on your Python version and operating system.

    if not ig_dir.exists():
        raise RuntimeError(
            "The given directory does not exist! Maybe the Python API is not enabled via cmake?")

    spec = importlib.util.spec_from_file_location(
        "ignis", ig_dir.joinpath("__init__.py"))
    module = importlib.util.module_from_spec(spec)
    sys.modules[spec.name] = module
    spec.loader.exec_module(module)

    return module

ignis = load_api()

In [ ]:
def get_doc(m: object):
    doc = inspect.getdoc(m)
    # return doc if doc is not None else ""
    return inspect.cleandoc(doc) if doc is not None else ""

def inspect_enum(e: enum.EnumType):
    return {
        "doc": get_doc(e),
        "members": list(e.__members__.keys())
    }

def inspect_property(p: property):
    return {
        "doc": get_doc(p),
    }

def split_methods(lines: str):
    it = iter(lines.splitlines())
    prev_line = next(it, None)
    for line in it:
        if line[0].isalpha():
            yield prev_line
            prev_line = line
        else:
            prev_line += line

    if prev_line:
        yield prev_line

def inspect_method(m: object):
    doc = get_doc(m)
    components = doc.split("\n\n")
    overloads = list(split_methods(components[0]))
    info = components[1:]
    if len(overloads) == 1:
        return [{
                "doc": "\n".join(info),
                "signature": overloads[0]
            }]
    else:
        signatures = []
        for o in overloads:
            doc_line = ""
            for i, line in enumerate(info):
                if o in line:
                    doc_line = info[i+1]
            signatures.append({
                "doc": doc_line,
                "signature": o
            })

        return signatures


def inspect_module(m: object):
    classes = {}
    methods = {}
    properties = {}
    enums = {}
    enum_members = []

    for name in dir(m):
        if name.startswith("__"):
            continue
        
        try:
            obj = getattr(m, name)
        except:
            continue
        
        if isinstance(obj, enum.EnumType):
            enums[name] = inspect_enum(obj)
        elif isinstance(obj, enum.Enum):
            enum_members.append(name)
        elif inspect.isclass(obj) or inspect.ismodule(obj):
            classes[name] = inspect_module(obj)
        elif isinstance(obj, property) or inspect.isgetsetdescriptor(obj):
            properties[name] = inspect_property(obj)
        else:
            methods[name] = inspect_method(obj)

    return {
        "doc": get_doc(m),
        "classes": classes,
        "methods": methods,
        "properties": properties,
        "enums": enums,
        "enum_members": enum_members
    }


inspect_module(ignis)

{'doc': 'Ignis python interface',
 'classes': {'BoundingBox': {'doc': 'BoundingBox',
   'classes': {},
   'methods': {'MakeEmpty': [{'doc': '',
      'signature': 'MakeEmpty() -> ignis.pyignis.BoundingBox'}],
    'MakeFull': [{'doc': '',
      'signature': 'MakeFull() -> ignis.pyignis.BoundingBox'}],
    'extend': [{'doc': '',
      'signature': 'extend(self, arg: ignis.pyignis.BoundingBox, /) -> ignis.pyignis.BoundingBox'},
     {'doc': '',
      'signature': "extend(self, arg: numpy.ndarray[dtype=float32, shape=(3), order='C'], /) -> ignis.pyignis.BoundingBox"}],
    'inflate': [{'doc': '',
      'signature': 'inflate(self, arg: float, /) -> None'}],
    'isInside': [{'doc': '',
      'signature': "isInside(self, arg: numpy.ndarray[dtype=float32, shape=(3), order='C'], /) -> bool"}],
    'isOverlapping': [{'doc': '',
      'signature': 'isOverlapping(self, arg: ignis.pyignis.BoundingBox, /) -> bool'}],
    'overlap': [{'doc': '',
      'signature': 'overlap(self, arg: ignis.pyignis.B